In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import upload_data, pull_model, pull_data
import catboost as cb
from sklearn.metrics import precision_score

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get the full Ethereum data from Hopsworks feature store.

In [2]:
eth_ohlc = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)

eth_ohlc

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.24s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.891200,0.921500,0.87500,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.906200,0.944700,0.89200,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.924900,1.030000,0.90580,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.990000,1.010000,0.94050,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.988700,1.010000,0.93750,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,2024-05-09,2977.000000,3055.000000,2958.00000,3036.000000,9.600000e+09,3.190231e+06,3.670000e+11,0.0,-0.018445,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3080,2024-05-10,3035.000000,3050.000000,2889.00000,2910.000000,1.060000e+10,3.557434e+06,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.000000,2935.000000,2894.00000,2911.000000,5.900000e+09,2.033385e+06,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3082,2024-05-12,2912.000000,2951.000000,2906.00000,2931.000000,5.200000e+09,1.779026e+06,3.581000e+11,1.0,0.010926,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


### Make Out-of-Training-Data Predictions starting at 1000 days, 100 days at a time (train model on 1000 days, then 1100, then 1200, etc, always testing on next 1000 days)

In [5]:
model = cb.CatBoostClassifier(iterations=300, learning_rate=.019, depth=6, verbose=False, random_seed=1)
predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count', 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']
response_var = 'tmw_1_0_percent_increase_binary'
other_vars = ['date', 'high', 'close', 'tmw_percent_increase_to_avg_high_low']

In [36]:
start = 1000
step = 100

all_predictions = []
for i in range(start, eth_ohlc.shape[0], step):

    train = eth_ohlc.iloc[0:i].copy()
    test = eth_ohlc.iloc[i:(i+step)].copy()

    model.fit(train[predictors_96], train[response_var])
    probability = model.predict_proba(test[predictors_96])[:,1]
    proba_series = pd.Series(probability, index=test.index, name="predicted_probability")
    predictions = pd.concat([test[other_vars], test[response_var], proba_series], axis=1)

    all_predictions.append(predictions)
    predictions_df = pd.concat(all_predictions)

In [37]:
predictions_df

,date,high,close,tmw_percent_increase_to_avg_high_low,tmw_1_0_percent_increase_binary,predicted_probability
1000,2018-08-11,334.260000,322.110000,0.006116,0.0,0.422378
1001,2018-08-12,328.590000,319.570000,-0.045514,0.0,0.248157
1002,2018-08-13,323.550000,286.500000,-0.013438,0.0,0.362669
1003,2018-08-14,286.370000,278.930000,0.050353,1.0,0.645693
1004,2018-08-15,303.590000,282.360000,0.038621,1.0,0.408482
...,...,...,...,...,...,...
3079,2024-05-09,3055.000000,3036.000000,-0.018445,0.0,0.389260
3080,2024-05-10,3050.000000,2910.000000,0.004467,0.0,0.459409
3081,2024-05-11,2935.000000,2911.000000,0.010306,1.0,0.382515
3082,2024-05-12,2951.000000,2931.000000,0.010926,1.0,0.354857


In [40]:
predictions_df['tmw_1_0_percent_increase_binary'] = predictions_df['tmw_1_0_percent_increase_binary'].astype('Int64')
predictions_df['date'] = pd.to_datetime(predictions_df['date']).dt.tz_localize(None)

# The last row is latest data (today), which does not have a prediction for tomorrow
predictions_df = predictions_df[:-1]
predictions_df

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,high,close,tmw_percent_increase_to_avg_high_low,tmw_1_0_percent_increase_binary,predicted_probability
1000,2018-08-11,334.26,322.11,0.006116,0,0.422378
1001,2018-08-12,328.59,319.57,-0.045514,0,0.248157
1002,2018-08-13,323.55,286.50,-0.013438,0,0.362669
1003,2018-08-14,286.37,278.93,0.050353,1,0.645693
1004,2018-08-15,303.59,282.36,0.038621,1,0.408482
...,...,...,...,...,...,...
3078,2024-05-08,3036.00,2975.00,0.023697,1,0.427190
3079,2024-05-09,3055.00,3036.00,-0.018445,0,0.389260
3080,2024-05-10,3050.00,2910.00,0.004467,0,0.459409
3081,2024-05-11,2935.00,2911.00,0.010306,1,0.382515


### Upload to Hopsworks

In [39]:
upload_data(predictions_df, 'eth_ohlc_predictions', 2)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/711829/fs/707652/fg/810342


Uploading Dataframe: 100.00% |██████████| Rows 2083/2083 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: eth_ohlc_predictions_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/711829/jobs/named/eth_ohlc_predictions_2_offline_fg_materialization/executions
